## imports

In [30]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

## Parameters

In [31]:
# Instantiate the model
input_size = 400  # Example input size
hidden_size = 64  # Example hidden size
num_layers = 2    # Example number of layers
num_features = 2  # Number of features in each twin

## Twin Rnn Model

In [32]:
class TwinRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_features):
        super(TwinRNN, self).__init__()
        
        # Define LSTM layers for twin 1 and twin 2
        self.lstmTwin1 = nn.LSTM(num_features, hidden_size, num_layers, batch_first=True)
        self.lstmTwin2 = nn.LSTM(num_features, hidden_size, num_layers, batch_first=True)
        
        # Initialize hidden states and cell states
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # Initialize the hidden state with the correct shape
        self.hidden1 = (torch.zeros(num_layers, 1, hidden_size),  # Hidden state
                        torch.zeros(num_layers, 1, hidden_size))  # Cell state

        self.hidden2 = self.init_hidden()
        
    def init_hidden(self):
        # Initialize hidden state and cell state
        return (torch.zeros(self.num_layers, 1, self.hidden_size),
                torch.zeros(self.num_layers, 1, self.hidden_size))
        
    def forward(self, x):
        # Twin 1
        xTwin1 = x[:, :, :2]
        features1 = xTwin1[:, :, 0].unsqueeze(-1)  # Selecting the first feature column
        inputsTwin1 = features1.float()
        inputsTwin1 = inputsTwin1.permute(1, 0, 2)  # Transposing dimensions for LSTM input
        lstm_out1, (h_n1, c_n1) = self.lstmTwin1(inputsTwin1, self.hidden1)
        featMapTwin1 = torch.cat([h_n1[i, :, :] for i in range(h_n1.shape[0])], dim=1)
        
        # Twin 2
        xTwin2 = x[:, :, 2:4]
        features2 = xTwin2
        features2 = xTwin2[:, :, 0].unsqueeze(-1)  # Selecting the first feature column
        inputsTwin2 = inputsTwin2.permute(1, 0, 2)
        lstm_out2, (h_n2, c_n2) = self.lstmTwin2(inputsTwin2, self.hidden2)
        featMapTwin2 = torch.cat([h_n2[i, :, :] for i in range(h_n2.shape[0])], dim=1)
        
        # Combine twins
        featMap = torch.cat((featMapTwin1, featMapTwin2), dim=1)
        
        return featMap



        


## Data

In [33]:
# Load the CSV file
data = pd.read_csv('./data.csv')

data.head()

,num1,num2,label
0,81,67,0
1,67,25,1
2,93,78,0
3,58,54,0
4,23,83,1


## Train

In [34]:

# Convert the DataFrame to a PyTorch tensor
data_tensor = torch.tensor(data.values, dtype=torch.float32)


model = TwinRNN(input_size, hidden_size, num_layers, num_features)

# Perform inference
with torch.no_grad():
    output = model(data_tensor.unsqueeze(0))  # Add batch dimension
    print(output)

RuntimeError: input.size(-1) must be equal to input_size. Expected 2, got 1

## import

In [35]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


## Hyperparameters

In [36]:
input_size = 1
hidden_size = 64
learning_rate = 0.001
num_epochs = 50
batch_size = 1

## Data

In [38]:

class CustomDataset(Dataset):
    def __init__(self, X1, X2, y):
        self.X1 = torch.tensor(X1, dtype=torch.float32)
        self.X2 = torch.tensor(X2, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)  # No need to modify the shape of labels

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X1[idx], self.X2[idx], self.y[idx]



# Load the CSV file
data = pd.read_csv("./data.csv")

# Extract features and labels
X1 = data['num1'].values.reshape(-1, 1)
X2 = data['num2'].values.reshape(-1, 1)
y = data['label'].values

# Normalize data
X1 = (X1 - np.mean(X1)) / np.std(X1)
X2 = (X2 - np.mean(X2)) / np.std(X2)


## Model

In [29]:
# Define the twin RRN model
class TwinRRN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(TwinRRN, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size)
        self.lstm2 = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size*2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x1, x2):
        _, (hidden1, _) = self.lstm1(x1.unsqueeze(1))
        _, (hidden2, _) = self.lstm2(x2.unsqueeze(1))
        combined_hidden = torch.cat((hidden1[-1], hidden2[-1]), dim=1)
        output = self.fc(combined_hidden)
        output = self.sigmoid(output)
        return output


ValueError: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.

## Train

In [ ]:

# Create dataset and dataloader
dataset = CustomDataset(X1, X2, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize model, loss, and optimizer
model = TwinRRN(input_size, hidden_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for x1, x2, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(x1, x2)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")


## Evaluate

In [39]:
with torch.no_grad():
    outputs = model(torch.tensor(X1, dtype=torch.float32), torch.tensor(X2, dtype=torch.float32))
    predictions = (outputs.numpy() > 0.5).astype(int)
    accuracy = np.mean(predictions.squeeze() == y)
    print(f"Accuracy: {accuracy}")


TypeError: TwinRNN.forward() takes 2 positional arguments but 3 were given